# Exploring eBay Car Sales Data

This project will be using a dataset of used cars from eBay Kleinanzeigen, a classified section of the German eBay website.

The main purpose of this project is to clean data and perform simple aggregate analysis on the price and mileage of the listed cars.

The original dataset is not available anymore. It was originally scraped and uploaded to Kaggle by user [orgesleka](https://www.kaggle.com/orgesleka).
However you can find a copy of it [here](https://data.world/data-society/used-cars-data) on the data.world website.

The version of the dataset we will be working on has been modified in the following ways:

* 50,000 data points from the full dataset (out of 370,000 data points) have been sampled by dataquest.io, to ensure the code runs quickly.
* The dataset has been dirtied a bit to more closely resemble what one would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with).


## Introduction 

The dataset include the following columns:

    dateCrawled - When this ad was first crawled. All field-values are taken from this date.
    name - Name of the car.
    seller - Whether the seller is private or a dealer.
    offerType - The type of listing
    price - The price on the ad to sell the car.
    abtest - Whether the listing is included in an A/B test.
    vehicleType - The vehicle Type.
    yearOfRegistration - The year in which the car was first registered.
    gearbox - The transmission type.
    powerPS - The power of the car in PS.
    model - The car model name.
    kilometer - How many kilometers the car has driven.
    monthOfRegistration - The month in which the car was first registered.
    fuelType - What type of fuel the car uses.
    brand - The brand of the car.
    notRepairedDamage - If the car has a damage which is not yet repaired.
    dateCreated - The date on which the eBay listing was created.
    nrOfPictures - The number of pictures in the ad.
    postalCode - The postal code for the location of the vehicle.
    lastSeenOnline - When the crawler saw this ad last online.


Now lets begin the cleaning and analysis by improting the libraries we need to work with.

In [1]:
import pandas as pd                                   
import numpy as np
import datetime as dt

In [2]:
autos = pd.read_csv('autos.csv', encoding = 'Latin-1') #reads the csv file into pandas

## Exploring the Data

Just from a brief glance we can see the number of rows the dataframe has, the data types, the columns names, and how many columns have null values in them.

We can even print out the first five rows to get a better visual understanding on how our data looks like in a table format.

In [3]:
print(autos.info())
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


We can make the following observations:

* The dataset contains 20 columns, most of which are strings.
* Some columns have null values, but none have more than ~20% null values.
* The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

## Cleaning the Data

Lets now start with the column names and rename them with a better  naming convention.

Instead of camel case we will rename the columns to follow snake case as for python that is the convention as per PEP8.

In [4]:
autos.rename(columns={'yearOfRegistration' : 'registration_year', 'monthOfRegistration' : 'registration_month', 'notRepairedDamage' : 'unrepaired_damage', 'dateCreated' : 'ad_created', 'dateCrawled' : 'date_crawled', 'offerType' : 'offer_type', 'vehicleType' : 'vehicle_type', 'powerPS' : 'power_ps', 'fuelType' : 'fuel_type', 'nrOfPictures' : 'n_pictures', 'postalCode' : 'postal_code', 'lastSeen' : 'last_seen'}, inplace = True)
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,n_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Now that the columns are in its proper naming convention, lets explore the data a bit further to get statistical overview of each of the coloumns.

In [5]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,n_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-21 20:37:19,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Running the above code we can see that the price and odometer coloumns are listed as strings since they are not returning any statistics and have letters and special characters in them. Below we'll be converting them to number formats so we can extract further information and handle the data better.

In [6]:
autos['price'] = autos['price'].str.replace('$','').str.replace(',','')
autos = autos.astype({'price': int})

autos['odometer'] = autos['odometer'].str.replace(',','').str.replace('km','')
autos = autos.astype({'odometer' : int})

<ipython-input-6-43f026748439>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  autos['price'] = autos['price'].str.replace('$','').str.replace(',','')


Once they are in a numerical format, lets further explore the price and odometer coloumn. 

In [7]:
print(autos['price'].describe().apply(lambda x: format(x, 'f')), '\n')
print(autos['odometer'].describe())

count       50000.000000
mean         9840.043760
std        481104.380500
min             0.000000
25%          1100.000000
50%          2950.000000
75%          7200.000000
max      99999999.000000
Name: price, dtype: object 

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer, dtype: float64


We can see that there are outliers in the price column since min is 0 and max is 99999999. 
Another good indicator is the mean value is only a fraction of what the standard deviation is for this column.

Lets now attempt to remove the outliers in the price column.
We can first remove all the values where the cars price is 0 since, they would defintely be sold at some cost on the website.

In [8]:
autos = autos[autos['price']!= 0]
autos.shape

(48579, 20)

After removing the 0 priced rows we are left with 48579 rows, i.e 1421 rows were removed.

Lets now check outliers closer to the max value.

In the world of cars surely there are going to be expensive ones that cost millions, but in order to understand more about where to draw the line and decide what cars are the outliers of the higher range lets explore the unique values of the price column.

In [9]:
#pd.options.display.max_rows = 4000 #enable this code to view the whole output
autos['price'].value_counts().sort_index()

1           156
2             3
3             1
5             2
8             1
           ... 
10000000      1
11111111      2
12345678      3
27322222      1
99999999      1
Name: price, Length: 2356, dtype: int64

From analysing the above data we can see that there is more of less even distribution amongst the price range coloumn. There are a 156 entries where the price is 1 and after 350,000 there is a huge spike in price. 

So lets remove all the rows where the price is 1 or where the price is above 350,000.

In [10]:
autos = autos[autos['price']!=1]
autos = autos[autos['price']<=350000]
print(autos['price'].describe().apply(lambda x: format(x, 'f')), '\n')

count     48409.000000
mean       5907.909707
std        9068.263463
min           2.000000
25%        1250.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: object 



Now there are 48409 rows remaining in our data set, before we continue lets rename the odometer column to make it easier to understand and work with.

In [11]:
autos.rename(columns={'odometer' : 'odometer_km'}, inplace = True)
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'n_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

## Exploring the date columns

There are 5 columns that should represent date values. We further explore the data columns involving dates:

* dateCrawled - When this ad was first crawled. All field-values are taken from this date.
* yearOfRegistration - The year in which the car was first registered.
* monthOfRegistration - The month in which the car was first registered.
* dateCreated - The date on which the eBay listing was created.
* lastSeenOnline - When the crawler saw this ad last online.

In [12]:
date_cols = ['date_crawled', 'last_seen', 'ad_created', 'registration_month', 'registration_year']

for col in date_cols:
    print("Data type of {} column is: {}".format(col,autos[col].dtype))

Data type of date_crawled column is: object
Data type of last_seen column is: object
Data type of ad_created column is: object
Data type of registration_month column is: int64
Data type of registration_year column is: int64


From running the for loop above we we can see:
The date_crawled, last_seen, and ad_created columns are of object type.

To get useful quantitative information out of these columns they will first need to be converted into datetime formats

In [13]:
def format_date(dataset,column):
    newformat = []
    for element in dataset[column]:
        element = dt.datetime.strptime(element, "%Y-%m-%d %H:%M:%S")
        element = element.date()
        newformat.append(element)
    dataset[column] = newformat   
    
format_date(autos,'date_crawled')
format_date(autos,'ad_created')
format_date(autos,'last_seen')

In [14]:
autos['date_crawled'].value_counts().sort_index() #Displays the value counts for date_crawled column sorted by date.

2016-03-05    1228
2016-03-06     681
2016-03-07    1745
2016-03-08    1611
2016-03-09    1600
2016-03-10    1559
2016-03-11    1578
2016-03-12    1789
2016-03-13     758
2016-03-14    1773
2016-03-15    1659
2016-03-16    1429
2016-03-17    1531
2016-03-18     625
2016-03-19    1682
2016-03-20    1830
2016-03-21    1806
2016-03-22    1594
2016-03-23    1562
2016-03-24    1423
2016-03-25    1528
2016-03-26    1561
2016-03-27    1507
2016-03-28    1687
2016-03-29    1652
2016-03-30    1633
2016-03-31    1540
2016-04-01    1633
2016-04-02    1718
2016-04-03    1868
2016-04-04    1766
2016-04-05     633
2016-04-06     153
2016-04-07      67
Name: date_crawled, dtype: int64

There are some drops on certain days, this could be attributed to lesser posts being available or something wrong fromt he crawlers side. But in any case it shouldnt be too much of an issue and shouldnt affect our analysis.

In [15]:
autos['ad_created'].value_counts().sort_index()

2015-06-11       1
2015-08-10       1
2015-09-09       1
2015-11-10       1
2015-12-05       1
              ... 
2016-04-03    1880
2016-04-04    1784
2016-04-05     571
2016-04-06     157
2016-04-07      60
Name: ad_created, Length: 76, dtype: int64

There is a sharp increase in the number of ads posted after 2016-03-04.

In [16]:
autos['last_seen'].value_counts().sort_index()

2016-03-05       52
2016-03-06      210
2016-03-07      262
2016-03-08      357
2016-03-09      466
2016-03-10      514
2016-03-11      599
2016-03-12     1152
2016-03-13      429
2016-03-14      611
2016-03-15      768
2016-03-16      796
2016-03-17     1359
2016-03-18      355
2016-03-19      766
2016-03-20      999
2016-03-21      998
2016-03-22     1035
2016-03-23      900
2016-03-24      956
2016-03-25      929
2016-03-26      814
2016-03-27      756
2016-03-28     1011
2016-03-29     1081
2016-03-30     1198
2016-03-31     1154
2016-04-01     1107
2016-04-02     1203
2016-04-03     1220
2016-04-04     1186
2016-04-05     6048
2016-04-06    10727
2016-04-07     6391
Name: last_seen, dtype: int64

The data in this column looks more or less uniform, not much analysis is required with this column.

Lets look further into the registration year column.

In [17]:
autos['registration_year'].describe()

count    48409.000000
mean      2004.774319
std         88.783278
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Because a car can't be registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate.

Since cars were only first officially invented in 1885 any registration years before that is being removed as well.

In [18]:
autos = autos[autos["registration_year"].between(1885,2016)]

In [19]:
print(autos['registration_year'].value_counts().sort_index(), '\n')
print(autos['registration_year'].describe())

1910       4
1927       1
1929       1
1931       1
1934       2
        ... 
2012    1310
2013     802
2014     662
2015     391
2016    1209
Name: registration_year, Length: 78, dtype: int64 

count    46531.000000
mean      2002.926049
std          7.159928
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64


Now from the above data we can see the distribution of registration years amongst the data. We can also see that the mean registration year is 2002 suggesting that mostly older used cars (around 10+ years old) are being sold in majority.

Lets focus now on aggregation of data for the brand coloumn. 

Lets try to find out the average price per brand listed on the eBay Kleinanzeigen website.

In [20]:
autos['brand'].describe()

count          46531
unique            40
top       volkswagen
freq            9834
Name: brand, dtype: object

We can see there are 40 unique brands and the top most mentioned brand is volswagen. This is not so surprising as the data is from the German ebay website and volswagen cars are from germany.

For the ease of analysis, lets only focus on the top 10 brands in terms of no. of cars listed on site for now.

In [21]:
top_10_brands = autos['brand'].value_counts().head(10).index
top_10_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat'],
      dtype='object')

In [22]:
autos = autos[autos.brand.isin(top_10_brands)]
autos['brand'].value_counts()

volkswagen       9834
bmw              5116
opel             4996
mercedes_benz    4488
audi             4030
ford             3256
renault          2193
peugeot          1390
fiat             1195
seat              850
Name: brand, dtype: int64

This table gives us a good indicator of popular used cars in the german markets.
We can see that volkswagens have the most listings while seat has the least listings on the website.

In [23]:
avg_brand_price = {}

for element in top_10_brands:
    avg_brand_price[element] = round(autos[autos['brand']==element].price.mean())

sorted(avg_brand_price.items(), key=lambda item: item[1]) #prints the dictonary we created sorted by its values


[('renault', 2484),
 ('fiat', 2818),
 ('opel', 2991),
 ('peugeot', 3101),
 ('ford', 3758),
 ('seat', 4413),
 ('volkswagen', 5418),
 ('bmw', 8367),
 ('mercedes_benz', 8657),
 ('audi', 9362)]

From the above data we can broadly classify these cars into 3 seperate categories:

* Cheap (<3000): renault, fiat, opel
* Mid-Range (3000 - 6000): peugeot, ford, seat, volkswagen
* Expensive (6000 - 10000): bmw, mercedes-benz, audi

Lets look further into the top 6 brands (in terms of value counts) and create a new dataframe (or table) that lets us compare each brands average price and average mileage.

In [24]:
top_6_brands = autos['brand'].value_counts().head(6).index #List of top 6 brands by value counts

top6_avg_bp = {}                                           #Empty dict to store the average brand price for each brand
                                                           #Key = Brand Name, Value = Avg Brand Price

for element in top_6_brands:
    top6_avg_bp[element] = avg_brand_price[element]        #Loop to create a dictionary of top 6 brands average prices.
    
avg_brand_price_series = pd.Series(top6_avg_bp)            #Converts the dictonary into a series

aggregate_df = pd.DataFrame(avg_brand_price_series, columns = ['avg_price']) #Converts the series into dataframe

In [25]:
top6_avg_brand_mileage = {}                                #Empty dict to store average mileage for each brand

for element in top_6_brands:                               #Loop to create a dictionary for the top 6 brands average mileage.
    top6_avg_brand_mileage[element] = round(autos[autos['brand']==element].odometer_km.mean())

sorted(top6_avg_brand_mileage.items(), key=lambda item: item[1]) #sorts and displays the top 6 brands average mileage.

[('ford', 124211),
 ('volkswagen', 128709),
 ('audi', 129208),
 ('opel', 129322),
 ('mercedes_benz', 130839),
 ('bmw', 132554)]

In [26]:
avg_brand_mileage_series = pd.Series(top6_avg_brand_mileage) #Coverts the dictionary into a series

aggregate_df['mileage'] = avg_brand_mileage_series #adds the series to the dataframe aggregate_df

aggregate_df

,avg_price,mileage
volkswagen,5418,128709
bmw,8367,132554
opel,2991,129322
mercedes_benz,8657,130839
audi,9362,129208
ford,3758,124211


From the above data we can see that mileage does not directly affect the average price across the brands.

In the case we were checking if mileage affects price within a particular brand, the outcome is likely that the older cars with more mileage would be sold for a cheaper price on average. 

In conclusion its difficult to say that mileage affects the average price of a particular brand.

However what this new dataframe does tell us is brands normally tend to hold their value and sell for resale prices that would be somewhat proportional to their initial costs straight out of the dealership.
As we can see all the brands have an average mileage within the range of 124,000kms to 132,000kms, this is not a large spread, since its less than 10,0000kms. However the prices for Audis are above 9000 whilst opels and fords cost less than half of that price for approximately the same mileage.